In [48]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import random
import numpy as np

import os
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Softmax
from sklearn.model_selection import train_test_split

### Genera training data

In [49]:
moves_converter = {"\ue012": "left", "\ue013": "up", "\ue014": "right", "\ue015": "down"}

In [57]:
def juego(): #funcion que juega 2048
    current_username = os.getlogin() #obtiene el nombre de usuario de la computadora

    driver_path_mac = "Users/davidesquer/Documents/chromedriverfolder/chromedrive"  #direccion del driver de chrome
    driver_path_windows = "C:/Users/David/Downloads/chromedriver_win32/chromedriver.exe" #direccion del driver de chrome

    if current_username == "David": #checa si el nombre de usuario es David
        driver_path = driver_path_windows #si es David, usa la direccion del driver de chrome para windows
    else:
        driver_path = driver_path_mac #si no es David, usa la direccion del driver de chrome para mac

    browser = webdriver.Chrome(executable_path=driver_path) #abre el navegador
    #options = webdriver.ChromeOptions()
    #options.add_argument("--headless")
    #options.add_argument("--incognito")
    browser.get("https://2048.io") #abre el juego
    time.sleep(2) #espera 5 segundos

    game_element = browser.find_element(By.TAG_NAME, "body") #elemento para mandar los movimientos

    # Define the possible moves
    moves = [Keys.UP, Keys.RIGHT, Keys.DOWN, Keys.LEFT] #lista de movimientos posibles

    score1 = browser.find_elements(By.XPATH, "/html/body/div/div[1]/div/div[1]") #elemento de score inicial
    si = score1[0].text #texto de score inicial
    si = si.split("\n")[0] #parsed score inicial

    tablas_total = [] #lista de todas las tablas de juego
    movimientos_total = [] #lista de todos los movimientos
    performances_total = [] #lista de todos los incrementos de score

    while True: #loop infinito, hasta que el juego termine
        l1 = [] #lista que contiene el estado del juego actual
        tiles = browser.find_elements(By.CSS_SELECTOR, ".tile") #estado del juego actual
        for tile in tiles: #loop sobre el estado del juego actual
            l1.append(tile.get_attribute("class").split()) #agrega el estado del juego actual a la lista l1
            
        board = np.array([[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]]) #matriz que contiene el estado del juego actual en ceros
        for i in range(len(l1)): #loop sobre el estado del juego actual
            value = int(l1[i][1].split("-")[1]) #valor de la casilla

            col = int(l1[i][2].split("-")[2])-1 #columna de la casilla
            row = int(l1[i][2].split("-")[3])-1 #fila de la casilla

            if board[row][col] < value: #si el valor de la casilla es mayor al valor de la casilla en la matriz, se actualiza el valor de la casilla en la matriz
                board[row][col] = value #actualiza el valor de la casilla en la matriz
        
        elements = browser.find_elements(By.CSS_SELECTOR, ".game-message") #elemento de mensaje de juego
        elements[0].text #texto de mensaje de juego

        if elements[0].text == "Game over!\nTry again": #si el juego termina, se rompe el loop
            break
        else:
            move = random.choice(moves) #elige un movimiento al azar
            game_element.send_keys(move) #ejecuta el movimiento elegido
            time.sleep(0.001)
        
        score = browser.find_elements(By.XPATH, "/html/body/div/div[1]/div/div[1]") #elemento de score final
        sf = score[0].text #texto de score final
        sf = sf.split("\n")[0] #parsed score final
        
        if len(tablas_total) == 0:
            lista = [board, move, (board.max()*board.sum())] #lista que contiene el estado del juego, el movimiento, y el incremento del score
            lista[1] = moves_converter[lista[1]] #convierte el movimiento a texto
        else:
            lista = [board, move, (board.max()*board.sum()) - (tablas_total[-1].sum()*tablas_total[-1].max())]
            lista[1] = moves_converter[lista[1]] #convierte el movimiento a texto

        tablas_total.append(board) #agrega a la lista las tablas de juego
        movimientos_total.append(lista[1]) #agrega a la lista los movimientos
        performances_total.append(lista[2]) #agrega a la lista los incrementos de score

    browser.close() #cierra el navegador

    return [tablas_total, movimientos_total, performances_total] #regresa la lista del juego completo

In [ ]:
tbl_tot = []
mov_tot = []
perf_tot = []

for i in range(100):
    d = juego()
    tablas_total = d[0]
    movimientos_total = d[1]
    performances_total = d[2]

    tbl_tot = tbl_tot + tablas_total
    mov_tot = mov_tot + movimientos_total
    perf_tot = perf_tot + performances_total
    print(i)

### Entrenamiento del modelo

In [ ]:
# Preprocess the dataset
def preprocess_data(board_states, moves, performances):
    # Encode board states as logarithm base 2 and normalize
    board_states = np.log2(np.maximum(board_states, 1)) / np.log2(2048)
    board_states = np.expand_dims(board_states, axis=-1)  # Add channel dimension for CNN
    
    # Map moves to integers
    move_mapping = {'up': 0, 'down': 1, 'left': 2, 'right': 3}
    moves = np.array([move_mapping[move] for move in moves])

    # One-hot encode moves
    moves = tf.keras.utils.to_categorical(moves, num_classes=4)
    
    return board_states, moves, performances

# Load your dataset
# X: board states, y_move: moves, y_performance: move performances
X, y_move, y_performance = tbl_tot, mov_tot, perf_tot

# Preprocess the dataset
X, y_move, y_performance = preprocess_data(X, y_move, y_performance)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(X, y_move, y_performance, test_size=0.2, random_state=42)

# Create the neural network model
model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(4, 4, 1)),
    Conv2D(128, kernel_size=(2, 2), activation='relu', padding='same'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

# Custom sample_weight for the performance of each move
sample_weight = w_train / np.max(w_train)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=100, sample_weight=sample_weight)

# Save the model
model.save('2048_model.h5')

### Test del modelo

In [7]:
board = np.array([[2,4,4,8], [2,8,8,2], [2,8,16,32], [2,2,1024,1024]]) #matriz que contiene el estado del juego actual en ceros

board_states = np.log2(np.maximum(board, 1)) / np.log2(2048)
test = np.expand_dims(board_states, axis=-1)  # Add channel dimension for CNN

In [21]:
board = np.array([[2,4,4,8], [2,8,8,2], [2,8,16,32], [2,2,1024,1024]]) #matriz que contiene el estado del juego actual en ceros
board_states = np.log2(np.maximum(board, 1)) / np.log2(2048) #normaliza el estado del juego
test = np.expand_dims(board_states, axis=-1) #agrega una dimension al estado del juego

input_data = np.expand_dims(test, axis=0)  # Nueva dimension: (1, 4, 4, 1)

predictions = model.predict(input_data) #obtiene las predicciones del modelo

best_move = np.argmax(predictions) #obtiene el indice del mejor movimiento

move_mapping_inverse = {0: 'up', 1: 'down', 2: 'left', 3: 'right'} #diccionario que mapea los movimientos a texto
best_move_string = move_mapping_inverse[best_move] #obtiene el texto del mejor movimiento

print("mejor movimiento:", best_move_string) #imprime el mejor movimiento

1/1 [==============================] - 0s 13ms/step
mejor movimiento: down


In [22]:
predictions

array([[0.00141408, 0.5362267 , 0.00330086, 0.45905834]], dtype=float32)

### Guarda training data

In [35]:
np.savez('2048_data.npz', tbl_tot, mov_tot, perf_tot) #guarda los datos del juego en un archivo .npz

In [43]:
loaded_data = np.load('2048_data.npz')

loaded_array_list = [loaded_data[key] for key in loaded_data]

loaded_data.close()

tbl_tot = loaded_array_list[0]
mov_tot = loaded_array_list[1]
perf_tot = loaded_array_list[2]